In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
Afum_endX <- Afum_endX %>%
    mutate(source = "Aspergillus")  # Adding the source label

QCreads_endX <- QCreads_endX %>%
    mutate(source = "QCreads")  # Adding the source label

combined_data <- bind_rows(Afum_endX, QCreads_endX) %>% select(-c("nsCount", "patient"))

data_tmp_RPM <- combined_data %>%
    filter(!grepl(sample_type, pattern = "wcDNA")) %>% 
    filter(origin == "IPA samples") %>% 
    filter(EndX %in% c("C", "G", "T", "A")) %>%
    group_by(sample, type, sample_type, sample_id, source, origin, read) %>% 
    pivot_wider(names_from = EndX, values_from = sCount, values_fill = 0) %>% 
    mutate(TotalCount = rowSums(across(c(C, T, G, A)), na.rm = TRUE)) %>% 
    reshape2::melt(variable.name = "EndX", value.name = "sCount") %>% 
    pivot_wider(names_from = source, values_from = sCount, values_fill = 0) %>% 
    mutate(RPM = Aspergillus/QCreads*10^6) %>% 
    select(-c("Aspergillus", "QCreads")) %>% 
    pivot_wider(names_from = EndX, values_from = RPM, values_fill = 0) %>% 
    mutate(across(c(C, T, G, A, "TotalCount"), ~ .x / TotalCount)) %>% 
    reshape2::melt(variable.name = "EndX", value.name = "RPM") %>% 
    mutate(EndX = ifelse(EndX == "TotalCount", yes = ".", no = as.character(EndX)))

stat.test <- compare_means(as.data.frame(data_tmp_RPM), 
                           formula = RPM~EndX, group.by = c("sample_type", "origin", "read"), 
                           method = "wilcox.test", alternative = "less", 
                           p.adjust.method = "bonferroni")

        sel_stat.test <- stat.test[which(stat.test$group2 == "."),]
        sel_stat.test$plot_pvalue <- round(sel_stat.test$p, digits = 3)
        sel_stat.test <- sel_stat.test %>% 
            mutate(loc_y = case_when(
                group1 == "A" ~ 5, 
                group1 == "C" ~ 5.2, 
                group1 == "G" ~ 5.4, 
                group1 == "T" ~ 5.6,
            )) %>% filter(p.signif != "ns")

RPM_boxplot_1nt <- data_tmp_RPM %>%
    mutate(EndX = factor(EndX, levels = rev(data_tmp_RPM$EndX  %>% unique()))) %>% 
    ggplot(aes(y = RPM, x = EndX, col = sample_type)) + 
                #geom_jitter(position = position_jitter(width = 0.15), size = 1, alpha = 0.5) +
                scale_color_manual(values = c("dsLP plasma" = "#C6896C", "ssLP plasma" = "#A84750", 
                                              "dsLP BAL" = "#7CA2C2", "ssLP BAL" = "#6066B6")) + 
                xlab("Selected end-motif") + ylab("Fold enrichment A. fumigatus RPM\ncompared to no end-motif selection") + 
                theme_bw() + 
                stat_pvalue_manual(sel_stat.test, y.position = "loc_y", coord.flip = TRUE,
                           label = "p.signif", remove.bracket = FALSE, tip.length = 0.01) +
                facet_grid(rows = vars(sample_type), 
                           cols = vars(read)) +
                theme(#axis.text.x = element_text(angle = 90, hjust = 1), 
                     strip.background = element_blank()) +  
                labs(color = "") + #ggtitle(tmp_title) + 
                coord_flip() +
                geom_dotplot(width = 0.15, binaxis = "y", stackdir = "center", dotsize = 0.2, fill = "white") + 
                geom_text(data = . %>% group_by(sample_type) %>%
                              summarise(unique_count = n_distinct(sample_id), .groups = "drop") %>%
                              mutate(label = paste("n = ", unique_count)), 
                      aes(x = Inf, y = Inf, label = label, col = sample_type), 
                      hjust = 1.5, vjust = 1.5, size = 3.5)

        options(repr.plot.width=14, repr.plot.height=8)
        RPM_boxplot_1nt %>% print()

        # Save the plot
        ggsave(paste0("../../output_figures/Aspergillus_RPM_EndMotif_1nt.png"), 
               plot = RPM_boxplot_1nt, width = 14, height = 8)
        ggsave(paste0("../../output_figures/Aspergillus_RPM_EndMotif_1nt.pdf"), 
               plot = RPM_boxplot_1nt, width = 14, height = 8)

In [ ]:
Afum_EndXX <- Afum_endXX %>%
    mutate(source = "Aspergillus")  # Adding the source label

QCreads_EndXX <- QCreads_endXX %>%
    mutate(source = "QCreads")  # Adding the source label

combined_data <- bind_rows(Afum_EndXX, QCreads_EndXX) %>% select(-c("nsCount", "patient"))

data_tmp_RPM <- combined_data %>%
    filter(!grepl(sample_type, pattern = "wcDNA")) %>% 
    filter(origin == "IPA samples") %>% 
    group_by(sample, type, sample_type, sample_id, source, origin, read) %>% 
    pivot_wider(names_from = EndXX, values_from = sCount, values_fill = 0) %>% 
    mutate(TotalCount = rowSums(
                across(c(CC,CG,GC,GT,CA,AT,AC,GA,TA,AG,AA,TT,TC,CT,TG,GG)))) %>% 
    reshape2::melt(variable.name = "EndXX", value.name = "sCount") %>% 
    pivot_wider(names_from = source, values_from = sCount, values_fill = 0) %>% 
    mutate(RPM = Aspergillus/QCreads*10^6) %>% 
    select(-c("Aspergillus", "QCreads")) %>% 
    pivot_wider(names_from = EndXX, values_from = RPM, values_fill = 0) %>% 
    mutate(across(c(CC,CG,GC,GT,CA,AT,AC,GA,TA,AG,AA,TT,TC,CT,TG,GG, TotalCount), ~ .x / TotalCount)) %>% 
    reshape2::melt(variable.name = "EndXX", value.name = "RPM") %>% 
    mutate(EndXX = ifelse(EndXX == "TotalCount", yes = "..", no = as.character(EndXX)))

stat.test <- compare_means(as.data.frame(data_tmp_RPM), 
                           formula = RPM~EndXX, group.by = c("sample_type", "origin", "read"), 
                           method = "wilcox.test", alternative = "less", 
                           p.adjust.method = "bonferroni")

        sel_stat.test <- stat.test[which(stat.test$group2 == ".."),]
        sel_stat.test$plot_pvalue <- round(sel_stat.test$p, digits = 3)
        sel_stat.test <- sel_stat.test %>% 
            mutate(loc_y = case_when(
                group1 == "AA" ~ 17.7, 
                group1 == "AC" ~ 17.0, 
                group1 == "AG" ~ 16.5,
                group1 == "AT" ~ 16.5, 
                group1 == "CA" ~ 15.5, 
                group1 == "CC" ~ 15.0,
                group1 == "CG" ~ 14.5, 
                group1 == "CT" ~ 14.0, 
                group1 == "GA" ~ 13.5,
                group1 == "GC" ~ 13.0, 
                group1 == "GG" ~ 12.5,
                group1 == "GT" ~ 12.0, 
                group1 == "TA" ~ 11.5, 
                group1 == "TC" ~ 11.0,
                group1 == "TG" ~ 10.5,
                group1 == "TT" ~ 10.0
            )) %>% filter(p.signif != "ns") 

RPM_boxplot_2nt <- data_tmp_RPM %>%
    mutate(EndXX = factor(EndXX, levels = rev(data_tmp_RPM$EndXX  %>% unique()))) %>% 
    ggplot(aes(y = RPM, x = EndXX, col = sample_type)) + 
                scale_color_manual(values = c("dsLP plasma" = "#C6896C", "ssLP plasma" = "#A84750", 
                                              "dsLP BAL" = "#7CA2C2", "ssLP BAL" = "#6066B6")) + 
                xlab("Selected end-motif") + ylab("Fold enrichment A. fumigatus RPM\ncompared to no end-motif selection") + 
                theme_bw() + 
                stat_pvalue_manual(sel_stat.test, y.position = "loc_y", coord.flip = TRUE,
                           label = "p.signif", remove.bracket = FALSE, tip.length = 0.01) +
                facet_grid(rows = vars(sample_type), 
                           cols = vars(read)) +
                theme(strip.background = element_blank()) +  
                labs(color = "") + 
                coord_flip() +
                geom_dotplot(width = 0.15, binaxis = "y", stackdir = "center", dotsize = 0.2, fill = "white") + 
                geom_text(data = . %>% group_by(sample_type, read) %>%
                              summarise(unique_count = n_distinct(sample_id), .groups = "drop") %>%
                              mutate(label = paste("n = ", unique_count)), 
                      aes(x = Inf, y = Inf, label = label, col = sample_type), 
                      hjust = 1.5, vjust = 1.5, size = 3.5) + 
                geom_text(data = . %>%
                            group_by(EndXX, sample_type, read) %>% 
                            summarise(mean_RPM = mean(RPM, na.rm = TRUE), .groups = 'drop') %>% 
                            mutate(label = round(mean_RPM, 2)), 
                         aes(x = EndXX, y = -1, label = label, col = sample_type), col = "black", size = 3, vjust = 0.5, hjust = 0.4)

        options(repr.plot.width=14, repr.plot.height=15)
        RPM_boxplot_2nt %>% print()

        # Save the plot
        ggsave(paste0("../../output_figures/Aspergillus_RPM_EndMotif_2nt.png"), 
               plot = RPM_boxplot_2nt, width = 14, height = 15)
        ggsave(paste0("../../output_figures/Aspergillus_RPM_EndMotif_2nt.pdf"), 
               plot = RPM_boxplot_2nt, width = 14, height = 15)

In [ ]:
Afum_EndXX <- Afum_endXX %>%
    mutate(source = "Aspergillus")  # Adding the source label

QCreads_EndXX <- QCreads_endXX %>%
    mutate(source = "QCreads")  # Adding the source label

combined_data <- bind_rows(Afum_EndXX, QCreads_EndXX) %>% select(-c("nsCount", "patient"))

data_tmp_RPM <- combined_data %>%
    filter(!grepl(sample_type, pattern = "wcDNA")) %>% 
    filter(origin == "IPA samples") %>% 
    group_by(sample, type, sample_type, sample_id, source, origin, read) %>% 
    pivot_wider(names_from = EndXX, values_from = sCount, values_fill = 0) %>% 
    mutate(TotalCount = rowSums(
                across(c(CC,CG,GC,GT,CA,AT,AC,GA,TA,AG,AA,TT,TC,CT,TG,GG)))) %>% 
    reshape2::melt(variable.name = "EndXX", value.name = "sCount") %>% 
    pivot_wider(names_from = source, values_from = sCount, values_fill = 0) %>% 
    mutate(RPM = Aspergillus/QCreads*10^6) %>% 
    select(-c("Aspergillus", "QCreads")) %>% 
    pivot_wider(names_from = EndXX, values_from = RPM, values_fill = 0) %>% 
    mutate(across(c(CC,CG,GC,GT,CA,AT,AC,GA,TA,AG,AA,TT,TC,CT,TG,GG, TotalCount), ~ .x / TotalCount)) %>% 
    reshape2::melt(variable.name = "EndXX", value.name = "RPM") %>% 
    mutate(EndXX = ifelse(EndXX == "TotalCount", yes = "..", no = as.character(EndXX)))

stat.test <- compare_means(as.data.frame(data_tmp_RPM), 
                           formula = RPM~EndXX, group.by = c("sample_type", "origin", "read"), 
                           method = "wilcox.test", alternative = "less", 
                           p.adjust.method = "bonferroni")

        sel_stat.test <- stat.test[which(stat.test$group2 == ".."),]
        sel_stat.test$plot_pvalue <- round(sel_stat.test$p, digits = 3)
        sel_stat.test <- sel_stat.test %>% 
            mutate(loc_y = case_when(
                group1 == "AA" ~ 17.6, 
                group1 == "AC" ~ 17.2, 
                group1 == "AG" ~ 16.8,
                group1 == "AT" ~ 16.4, 
                group1 == "CA" ~ 16.0, 
                group1 == "CC" ~ 15.6,
                group1 == "CG" ~ 15.2, 
                group1 == "CT" ~ 14.8, 
                group1 == "GA" ~ 14.4,
                group1 == "GC" ~ 14.0, 
                group1 == "GG" ~ 13.6,
                group1 == "GT" ~ 13.2, 
                group1 == "TA" ~ 12.8, 
                group1 == "TC" ~ 12.4,
                group1 == "TG" ~ 12.0,
                group1 == "TT" ~ 11.6
            )) %>% filter(p.signif != "ns") 

GC_RPM_boxplot_2nt <- data_tmp_RPM %>%
    filter(read == "R1") %>% 
    filter(EndXX %in% c("CG","..")) %>% 
    mutate(EndXX = factor(EndXX, levels = rev(data_tmp_RPM$EndXX  %>% unique()))) %>% 
    ggplot(aes(y = RPM, x = EndXX, col = sample_type)) + 
                scale_color_manual(values = c("dsLP plasma" = "#C6896C", "ssLP plasma" = "#A84750", 
                                              "dsLP BAL" = "#7CA2C2", "ssLP BAL" = "#6066B6")) + 
                xlab("Selected end-motif") + ylab("Fold enrichment in RPM\ncompared to no end-motif selection") + 
                theme_bw() + 
                stat_pvalue_manual(sel_stat.test %>% filter(group1 == "CG") %>% filter(read == "R1"), 
                                   y.position = "loc_y", 
                           label = "p.signif", remove.bracket = FALSE, tip.length = 0.01) +
                facet_grid(rows = vars(sample_type), 
                           cols = vars(read)) +
                theme(
                     strip.background = element_blank()) +  
                labs(color = "") + 
                geom_dotplot(width = 0.5, binaxis = "y", stackdir = "center", dotsize = 1, fill = "white") + 
                geom_text(data = . %>%
                            filter(read == "R1") %>% group_by(EndXX, sample_type) %>% filter(EndXX == "CG") %>% 
                            summarise(mean_RPM = mean(RPM, na.rm = TRUE), .groups = 'drop') %>% 
                            mutate(label =round(mean_RPM, 2)), 
                         aes(x = "CG", y = 0, label = label, col = sample_type), size = 4, vjust = 0, hjust = -0.5)

        options(repr.plot.width=6, repr.plot.height=8)
        GC_RPM_boxplot_2nt %>% print()

        # Save the plot
        ggsave(paste0("../../output_figures/Aspergillus_RPM_GC_EndMotif_2nt.png"), 
               plot = GC_RPM_boxplot_2nt, width = 6, height = 8)
        ggsave(paste0("../../output_figures/Aspergillus_RPM_GC_EndMotif_2nt.pdf"), 
               plot = GC_RPM_boxplot_2nt, width = 6, height = 8)

In [ ]:
data_tmp_RPM %>%
    filter(read == "R1") %>% 
    filter(EndXX == "CG") %>% 
    summarize(mean_RPM = mean(RPM, na.rm = TRUE))

data_tmp_RPM %>%
    filter(read == "R1") %>% 
    filter(EndXX == "CG") %>% 
    group_by(sample_type) %>% 
    group_by(sample_type) %>% 
    summarize(mean_RPM = mean(RPM, na.rm = TRUE))

In [ ]:
prevalence1 <- Afum_EndXX %>%
    group_by(origin, sample_type, sample, type, read) %>%
    summarize(prevalence1 = n_distinct(sample_id),  
              .groups = 'drop')

prevalence2 <- Afum_EndXX %>%
    filter(EndXX == "CG") %>%
    group_by(origin, sample_type, sample, type, read) %>%
    summarize(prevalence2 = n_distinct(sample_id),  
              .groups = 'drop')

prevalence_ratio <- prevalence1 %>%
    left_join(prevalence2, by = c("origin", "sample_type", "sample", "type", "read")) %>%
    mutate(ratio = prevalence2 / prevalence1) 

prevalence_plot_2nt <- prevalence_ratio %>% 
    filter(origin == "IPA samples") %>% 
    filter(read == "R1") %>% 
    ggplot(aes(x = "CG", y = ratio, fill = sample_type)) + 
        geom_bar(position=position_dodge(width=0.1), stat="identity", alpha = 0.5) + 
        facet_grid(rows = vars(sample_type), 
                           cols = vars(read)) + 
        theme_bw() + theme(panel.grid.major = element_blank(),
                           panel.grid.minor = element_blank(),
                           strip.background = element_blank()) + 
        scale_fill_manual(values = c("dsLP plasma" = "#C6896C", "ssLP plasma" = "#A84750", 
                                      "dsLP BAL" = "#7CA2C2", "ssLP BAL" = "#6066B6")) + 
        ylab("Prevalence") + labs(fill = NULL)

options(repr.plot.width=3, repr.plot.height=8)
prevalence_plot_2nt %>% print()

# Save the plot
ggsave(paste0("../../output_figures/Aspergillus_prevalence_GC_EndMotif_2nt.png"), 
    plot = prevalence_plot_2nt, width = 3, height = 8)
ggsave(paste0("../../output_figures/Aspergillus_prevalence_GC_EndMotif_2nt.pdf"), 
    plot = prevalence_plot_2nt, width = 3, height = 8)

In [ ]:
options(repr.plot.width=3, repr.plot.height=8)
percentage_plot_2nt <- QCreads_endXX %>% 
    filter(!grepl(sample_type, pattern = "wcDNA")) %>% 
    group_by(origin, sample_type) %>%
            mutate(unique_sample_ids = n_distinct(sample_id)) %>% 
            mutate(label_new = paste0(origin, "\n(n = ", unique_sample_ids, ")")) %>% 
    group_by(label_new, sample_id, sample_type, sample, type, patient, origin, read) %>% 
    filter(origin == "IPA samples") %>% 
    mutate(nsCount = sCount/sum(sCount)*100) %>% 
    filter(EndXX == "CG")  %>% 
    filter(read == "R1") %>% 
    mutate(sample_type = factor(sample_type, levels = c("dsLP BAL", "dsLP plasma", "ssLP BAL" ,"ssLP plasma"))) %>%
    mutate(label_new = factor(label_new, levels = c('IPA samples\n(n = 10)', 'IPA samples\n(n = 4)', 
                                                      'IPA samples\n(n = 11)', 'IPA samples\n(n = 5)'))) %>% 
    ggplot(aes(x = "CG", y = nsCount, col = label_new)) + 
        geom_boxplot() + 
        geom_point(alpha = 0.5) +
        facet_grid(rows = vars(label_new), 
                        cols = vars(read)) + 
        theme_bw() + theme(strip.background = element_blank()) + 
        scale_color_manual(values = c('IPA samples\n(n = 4)' = "#C6896C", 'IPA samples\n(n = 5)' = "#A84750", 
                                      'IPA samples\n(n = 10)' = "#7CA2C2", 'IPA samples\n(n = 11)' = "#6066B6")) + 
        ylab("Percentage reads") + ylim(0,3) + 
        geom_text(data = . %>%  group_by(label_new, sample_type) %>%
                summarise(mean_nsCount = mean(nsCount), .groups = 'drop') %>% 
                mutate(label = round(mean_nsCount, digits = 1)), 
                aes(x = "CG", y = Inf, label = label, col = label_new), 
                     vjust = 1.5, size = 3.5)

options(repr.plot.width=3, repr.plot.height=8)
percentage_plot_2nt %>% print()

# Save the plot
ggsave(paste0("../../output_figures/Aspergillus_percReads_GC_EndMotif_2nt.png"), 
    plot = percentage_plot_2nt, width = 3, height = 8)
ggsave(paste0("../../output_figures/Aspergillus_percReads_GC_EndMotif_2nt.pdf"), 
    plot = percentage_plot_2nt, width = 3, height = 8)        